# Simplified Kraken Sequence Classification Algorithm

**Name: Shayan Aryania **

**Student Number: 402211767 **

## Introduction

### Overview
Sequence classification is a fundamental task in bioinformatics, pivotal for understanding biological data derived from sequencing technologies. It involves determining the taxonomic origin of sequences extracted from samples, which can range from single cells to complex environments like soil or ocean water. This classification process is crucial for tasks such as identifying pathogens, studying biodiversity, or understanding microbial communities.

### The Kraken Algorithm
Kraken is a bioinformatics algorithm designed for the rapid and accurate classification of metagenomic sequences using exact k-mer matching. The original Kraken algorithm leverages a sophisticated database structure to map short DNA sequences (k-mers) extracted from longer genomic sequences to the lowest common ancestor (LCA) in a taxonomic tree of all genomes that contain them. This mapping allows the algorithm to classify sequences with high precision based on the taxonomic information of their constituent k-mers.

# Database Preparation

## Downloading the `Kraken2`'s `Viral` Database

In [1]:
!wget "https://genome-idx.s3.amazonaws.com/kraken/k2_viral_20240112.tar.gz" -P /content/viral-k2/

--2024-05-11 21:49:06--  https://genome-idx.s3.amazonaws.com/kraken/k2_viral_20240112.tar.gz
Resolving genome-idx.s3.amazonaws.com (genome-idx.s3.amazonaws.com)... 52.217.73.140, 52.217.235.121, 52.217.229.9, ...
Connecting to genome-idx.s3.amazonaws.com (genome-idx.s3.amazonaws.com)|52.217.73.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 506061528 (483M) [application/x-tar]
Saving to: ‘/content/viral-k2/k2_viral_20240112.tar.gz’

k2_viral_20240112.t 100%[===================>] 482.62M  28.5MB/s    in 51s     

2024-05-11 21:49:57 (9.55 MB/s) - ‘/content/viral-k2/k2_viral_20240112.tar.gz’ saved [506061528/506061528]



## Extracting the Database

In [2]:
!tar xfvz "/content/viral-k2/k2_viral_20240112.tar.gz" -C '/content/viral-k2/'

hash.k2d
opts.k2d
taxo.k2d
seqid2taxid.map
inspect.txt
ktaxonomy.tsv
library_report.tsv
database100mers.kmer_distrib
database150mers.kmer_distrib
database200mers.kmer_distrib
database250mers.kmer_distrib
database300mers.kmer_distrib
database50mers.kmer_distrib
database75mers.kmer_distrib


## Downloading genome sequences

In [3]:
import pandas as pd

k2_viral_df = pd.read_csv('/content/viral-k2/library_report.tsv', sep='\t')
k2_viral_df['accession_id'] = k2_viral_df['Sequence Name'].apply(lambda x: x.split()[0][1:])

k2_viral_df = k2_viral_df.sort_values('accession_id')

print(k2_viral_df.shape)

(18639, 4)


In [4]:
k2_viral_df.head()

,#Library,Sequence Name,URL,accession_id
15700,viral,">AC_000001.1 Ovine adenovirus A, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000001.1
8372,viral,">AC_000002.1 Bovine adenovirus B, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000002.1
1050,viral,">AC_000003.1 Canine adenovirus 1, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000003.1
12390,viral,">AC_000004.1 Duck adenovirus A, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000004.1
5103,viral,">AC_000005.1 Human mastadenovirus A, complete ...",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000005.1


### Downloading `viral.1.1.genomic.fna.gz` from NCBI

In [5]:
!wget https://ftp.ncbi.nlm.nih.gov/refseq/release/viral/viral.1.1.genomic.fna.gz -P /content/viral-ncbi/

--2024-05-11 21:50:05--  https://ftp.ncbi.nlm.nih.gov/refseq/release/viral/viral.1.1.genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.10, 130.14.250.11, 2607:f220:41e:250::11, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168654208 (161M) [application/x-gzip]
Saving to: ‘/content/viral-ncbi/viral.1.1.genomic.fna.gz’

viral.1.1.genomic.f 100%[===================>] 160.84M  48.7MB/s    in 3.8s    

2024-05-11 21:50:09 (42.1 MB/s) - ‘/content/viral-ncbi/viral.1.1.genomic.fna.gz’ saved [168654208/168654208]



In [6]:
!gunzip /content/viral-ncbi/viral.1.1.genomic.fna.gz

In [7]:
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 16.7 MB/s eta 0:00:00


In [8]:
from Bio import SeqIO

# Path to the extracted FASTA file
fasta_path = '/content/viral-ncbi/viral.1.1.genomic.fna'

# Load the FASTA file using SeqIO.parse
sequences = list(SeqIO.parse(fasta_path, "fasta"))
len(sequences)

18723

Removing sequences which are not present in the `k2 viral` database:

In [9]:
k2_viral_ids = k2_viral_df['accession_id'].to_list()

In [10]:
sequences = [seq_record for seq_record in sequences if seq_record.id in k2_viral_ids]
len(sequences)

18639

In [11]:
sequences = sorted(sequences, key=lambda x: x.id)

Adding `seq` column to the `k2_viral_df`:

In [12]:
k2_viral_df['seq'] = [str(seq_record.seq) for seq_record in sequences]

In [13]:
k2_viral_df.head()

,#Library,Sequence Name,URL,accession_id,seq
15700,viral,">AC_000001.1 Ovine adenovirus A, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000001.1,CATCATCAATAATATACGGTGCATTTTGTGCGTGATGACGTATACA...
8372,viral,">AC_000002.1 Bovine adenovirus B, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000002.1,CATCATCAATAATCTACAGTACACTGATGGCAGCGGTCCAACTGCC...
1050,viral,">AC_000003.1 Canine adenovirus 1, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000003.1,CATCATCAATAATATACAGGACAAAGAGGTGTGGCCTAAATGTTGT...
12390,viral,">AC_000004.1 Duck adenovirus A, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000004.1,CTCATGTCATTAATAAGACCATGCAGAAAATGCAAATGAGGCGAAG...
5103,viral,">AC_000005.1 Human mastadenovirus A, complete ...",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000005.1,CCTATCTAATAATATACCTTATACTGGACTAGTGCCAATATTAAAA...


# Downloading the taxonomy tree

In [14]:
!wget ftp://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdump.tar.gz
!tar -xzvf taxdump.tar.gz nodes.dmp names.dmp

--2024-05-11 21:51:00--  ftp://ftp.ncbi.nlm.nih.gov/pub/taxonomy/taxdump.tar.gz
           => ‘taxdump.tar.gz’
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.7, 130.14.250.10, 2607:f220:41e:250::11, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.7|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/taxonomy ... done.
==> SIZE taxdump.tar.gz ... 65175473
==> PASV ... done.    ==> RETR taxdump.tar.gz ... done.
Length: 65175473 (62M) (unauthoritative)

taxdump.tar.gz      100%[===================>]  62.16M  36.3MB/s    in 1.7s    

2024-05-11 21:51:02 (36.3 MB/s) - ‘taxdump.tar.gz’ saved [65175473]

names.dmp
nodes.dmp


Use the given `taxonomy` module to load the tree. You can refer to the `https://github.com/frallain/NCBI_taxonomy_tree` to see more details about this module and how to work with:

In [15]:
from taxonomy import NcbiTaxonomyTree

taxonomy_tree = NcbiTaxonomyTree(nodes_filename="./nodes.dmp", names_filename="./names.dmp")

## `accession_id` to `tax_id` mapping

As you probably noticed, the taxonomy tree uses `tax_id` to identify the nodes. However, the `k2_viral_df` uses `accession_id` to identify the genomes. We need to create a mapping between these two identifiers.

The cell below may take a while to run!

In [16]:
!wget ftp://ftp.ncbi.nih.gov/pub/taxonomy/accession2taxid/nucl_gb.accession2taxid.gz
!gunzip -f nucl_gb.accession2taxid.gz

--2024-05-11 21:52:00--  ftp://ftp.ncbi.nih.gov/pub/taxonomy/accession2taxid/nucl_gb.accession2taxid.gz
           => ‘nucl_gb.accession2taxid.gz’
Resolving ftp.ncbi.nih.gov (ftp.ncbi.nih.gov)... 130.14.250.11, 130.14.250.12, 2607:f220:41e:250::7, ...
Connecting to ftp.ncbi.nih.gov (ftp.ncbi.nih.gov)|130.14.250.11|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/taxonomy/accession2taxid ... done.
==> SIZE nucl_gb.accession2taxid.gz ... 2407649552
==> PASV ... done.    ==> RETR nucl_gb.accession2taxid.gz ... done.
Length: 2407649552 (2.2G) (unauthoritative)

nucl_gb.accession2t 100%[===================>]   2.24G  47.9MB/s    in 48s     

2024-05-11 21:52:49 (47.8 MB/s) - ‘nucl_gb.accession2taxid.gz’ saved [2407649552]



In [17]:
accession_list = k2_viral_df['accession_id'].to_list()

with open('accession_list.txt', 'w') as file:
    for accession in accession_list:
        file.write(accession + '\n')

The cell below may take a while to run!

In [18]:
!grep -F -f accession_list.txt nucl_gb.accession2taxid > filtered_accession2taxid.txt

In [19]:
from tqdm import tqdm

def parse_accession_to_taxid(filename):
    accession_to_taxid = {}
    with open(filename, 'r') as file:
        for line in tqdm(file, total=18639):
            parts = line.strip().split('\t')
            accession = parts[1]
            taxid = parts[2]
            accession_to_taxid[accession] = taxid
    return accession_to_taxid

accession_tax_mapping = parse_accession_to_taxid('filtered_accession2taxid.txt')

100%|██████████| 18639/18639 [00:00<00:00, 652007.31it/s]


In [20]:
accession_tax_mapping = dict(sorted(accession_tax_mapping.items()))

k2_viral_df['tax_id'] = accession_tax_mapping.values()

In [21]:
k2_viral_df.head()

,#Library,Sequence Name,URL,accession_id,seq,tax_id
15700,viral,">AC_000001.1 Ovine adenovirus A, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000001.1,CATCATCAATAATATACGGTGCATTTTGTGCGTGATGACGTATACA...,114424
8372,viral,">AC_000002.1 Bovine adenovirus B, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000002.1,CATCATCAATAATCTACAGTACACTGATGGCAGCGGTCCAACTGCC...,129950
1050,viral,">AC_000003.1 Canine adenovirus 1, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000003.1,CATCATCAATAATATACAGGACAAAGAGGTGTGGCCTAAATGTTGT...,10512
12390,viral,">AC_000004.1 Duck adenovirus A, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000004.1,CTCATGTCATTAATAAGACCATGCAGAAAATGCAAATGAGGCGAAG...,130328
5103,viral,">AC_000005.1 Human mastadenovirus A, complete ...",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000005.1,CCTATCTAATAATATACCTTATACTGGACTAGTGCCAATATTAAAA...,129875


# SimpleKraken

We strongly recommend you to read the **Kraken** paper before you start implementing the **SimpleKraken** algorithm. You can find the paper [here](https://genomebiology.biomedcentral.com/articles/10.1186/gb-2014-15-3-r46).

## 1. Extracting K-mers

Kraken uses Jellyfish to extract k-mers from the reference database. You can read the exact implementation details in the Kraken paper. In this tutorial, we will use a simplified version of the k-mer extraction process.

In [22]:
def extract_kmers(sequence, k):
    # TODO: implement a simple function that extracts all k-mers from the given sequence
  kmers = []
  for i in range(len(sequence) - k + 1):
    kmer = sequence[i:i+k]
    kmers.append(kmer)
  return kmers


## 2. Building The Database

Calculate the number of appearances of each k-mer in the given set of genomes, and only keep k-mers that appear more than a threshold `t`.


You can use the `Jellyfish` to calculate the number of appearances of the k-mers efficiently:

In [23]:
!apt install jellyfish

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libhts3 libhtscodecs2 libjellyfish-2.0-2
The following NEW packages will be installed:
  jellyfish libhts3 libhtscodecs2 libjellyfish-2.0-2
0 upgraded, 4 newly installed, 0 to remove and 45 not upgraded.
Need to get 1,283 kB of archives.
After this operation, 3,550 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libhtscodecs2 amd64 1.1.1-3 [53.2 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libhts3 amd64 1.13+ds-2build1 [390 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libjellyfish-2.0-2 amd64 2.3.0-12ubuntu2 [65.0 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 jellyfish amd64 2.3.0-12ubuntu2 [775 kB]
Fetched 1,283 kB in 1s (1,398 kB/s)
Selecting previously unselected package libhtscodecs2:amd64.
(Reading database ... 121918 files

Run the `jellyfish` with the proper arguments:

In [24]:
def fasta(df, filename):
    with open(filename, 'w') as f:
        for index, row in df.iterrows():
            f.write('>sequence'+ str(index) + '|kraken:taxid|' + str(row['tax_id']) + ' \n')
            f.write(str(row['seq']) + '\n')

In [25]:
k2_viral_df = k2_viral_df.sample(n=800, random_state=42)

In [26]:
k2_viral_df = k2_viral_df.sample(n=800, random_state=42)
fasta(k2_viral_df, 'kraken.fasta')

In [27]:
# TODO
!jellyfish count -m 36 -s 100M -t 10 -C -o output.jf kraken.fasta

Parse the result file to get a list of suitable k-mers:

In [28]:
!jellyfish dump -L 1 output.jf > filtered_kmers.txt


Build a database (a `dict`) that assigns each k-mer in the `filtered_kmers` to a list of all genome IDs that have this k-mer:

In [29]:
# TODO
def filterd_kmer(file_path):
    kmers = []
    with open(file_path, "r") as file:
        while True:
            count_line = file.readline()
            if not count_line:
                break
            kmer_line = file.readline()
            if not kmer_line:
                break
            kmer = kmer_line.strip('\n')
            kmers.append(kmer)
    return kmers

In [30]:
filtered_kmers = filterd_kmer("filtered_kmers.txt")

In [31]:
def extract_kmers(sequence, k):
    # TODO: implement a simple function that extracts all k-mers from the given sequence
  kmers = []
  for i in range(len(sequence) - k + 1):
    kmer = sequence[i:i+k]
    kmers.append(kmer)
  return kmers

In [32]:
def build_database(sequences, k):
    database = defaultdict(list)
    # TODO
    for sequence in tqdm(sequences):
      kmers = extract_kmers(sequence,k)
      for kmer in kmers:
        if kmer in filtered_kmers_set:
          database[kmer].append(sequence_to_taxid[sequence])
    return database

In [33]:
from collections import defaultdict

In [34]:
sequence_to_taxid = dict(zip(k2_viral_df['seq'], k2_viral_df['tax_id']))
filtered_kmers_set = set(filtered_kmers)

In [35]:
database = build_database(k2_viral_df['seq'],36)

100%|██████████| 800/800 [01:34<00:00,  8.45it/s]


## 3. Taxonomy tree and Lowest Common Ancestor (LCA)

Using the database created in the previous step, and the given taxonomy tree, for each k-mer in the genomes, find the LCA of those genomes on the taxonomy tree. Create a dictionary for each internal node in the tree that contains the minimizer of the k-mer and maps it to the k-mer:

$$node_i = \{'minimizer1': 'k-mer'\}$$

In [36]:
def lcafinding(tree, id):
    if isinstance(id, str):
        id = [int(id)]
    tax = id[0]
    path = [int(tax)]
    while tree.getParent([int(tax)])[int(tax)] is not None:
        tax = tree.getParent([int(tax)])[int(tax)]
        path.append(int(tax))
    for different in id[1:]:
        while different is not None:
            if different in path:
                ind = path.index(different)
                path = path[ind:]
            different = tree.getParent([int(different)])[int(different)]
    return path[0]

In [37]:
def node_dictionary(tree, database):
    node_dict = defaultdict(lambda: defaultdict(list))
    for kmer, id in database.items():
        lca = lcafinding(tree, id)
        kmers = extract_kmers(kmer, 13)
        minimizer = min(kmers)
        node_dict[minimizer][kmer].append(lca)
    return node_dict

In [38]:
node_dict = node_dictionary(taxonomy_tree, database)

In [39]:
kmercount = defaultdict(list)
for minimizer, kmer in node_dict.items():
      kmercount[minimizer].append(len(kmer))

## 4. Scoring and Decision Making

To search and score a given genome $g$ through the KDB, we first create the k-mers of the $g$, and then calculate their minimizers and count the number of sequential k-mers $s$ that have that minimizer. Then for each k-mer we search through KDB to find the candidate internal nodes that have the corresponding minimizer in their keys. Then we use exact match through those k-mer values to find the unique internal node and assigns the $s$ to that node. At the end we do a root to leaf traverse and aggregate the scores of parents to calculate leaf scores. The leaf with the highest score is the target.


In [46]:
# TODO
def score(gen, dictionary):
    kmers = extract_kmers(gen, 36)
    node_sum = defaultdict(int)
    for kmer in kmers:
        minimizer = min(extract_kmers(kmer, 13))
        kmer_lca = dictionary.get(minimizer, -1)
        if kmer_lca != -1 and kmer in kmer_lca.keys():
            temp_lca = kmer_lca[kmer][0]
            node_sum[temp_lca] += 1
    root = temp_lca
    while True:
        previous = root
        root = taxonomy_tree.getParent([previous])[previous]
        if root is None:
            break
    root = previous
    nodes = node_sum.keys()

    def targetfinding(node, target, path=None):
        if path is None:
            path = []
        if node == target:
            path.append(node)
            return path
        for child in taxonomy_tree.getChildren([node])[node]:
            result = targetfinding(child, target, path)
            if result is not None:
                result.insert(0, node)
                return result
        if node in path:
            path.remove(node)
        return None
    max_score = 0
    result = None
    for node in nodes:
        path = targetfinding(root, node)
        scores = 0
        for item in path:
            scores += node_sum.get(item,0)
        if scores > max_score:
            max_score = scores
            result = node
    return result

# Evaluation

Generate a few random genomes from the `k2_viral_df` and classify them using the `SimpleKraken` algorithm. Compare the result with the actual taxonomy of the genome.
You can also use a read of a genome and mutate it (e.g. poisson mutation) to see how the algorithm performs in a more realistic scenario:

In [42]:
import random

In [47]:
random = k2_viral_df.sample(n=20, random_state=42)
outputs = []
for _, row in random.iterrows():
    genome = row['seq']
    output = score(genome, node_dict)
    outputs.append(output)
    print(f"ID = {row['tax_id']}, Match = {output}")

ID = 1678231, Match = 1678231
ID = 1923455, Match = 1923455
ID = 412969, Match = 412969
ID = 1980433, Match = 1980433
ID = 356114, Match = 356114
ID = 2664941, Match = 2664941
ID = 419435, Match = 419435
ID = 2202565, Match = 2202565
ID = 2152567, Match = 2152567
ID = 134394, Match = 134394
ID = 2420320, Match = 2420320
ID = 2847998, Match = 2847998
ID = 1708654, Match = 1708654
ID = 2681608, Match = 2681608
ID = 1772297, Match = 1772297
ID = 2786836, Match = 2786836
ID = 1851087, Match = 1851087
ID = 2304514, Match = 2304514
ID = 1739967, Match = 1739967
ID = 31658, Match = 31658


# Bonus

It's worth mentioning that you can use the `kraken2` tool to classify sequences using the database we downloaded in the first section. The `kraken2` tool is a more sophisticated implementation of the Kraken algorithm that uses a database structure optimized for fast and accurate classification of metagenomic sequences.

I'll put a few cells below which will help you install the `kraken2` tool and you can play around with it to classify sequences using the downloaded database.

You can find more information about the `kraken2` tool in the official [GitHub repository](https://github.com/DerrickWood/kraken2/wiki/Manual).

NOTE: The session will be restarted after running the cell below:

In [48]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:22
🔁 Restarting kernel...


In [ ]:
import condacolab
condacolab.check()

In [2]:
!conda install bioconda::kraken2

Channels:
 - conda-forge
 - bioconda
Platform: linux-64
Solving environment: | / - \ | done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.4.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - bioconda::kraken2


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    blast-2.15.0               | pl5321h6f7f691_1       146.0 MB  bioconda
    ca-certificates-2024.2.2   |       hbcca054_0         152 KB  conda-forge
    certifi-2024.2.2           |     pyhd8ed1ab_0         157 KB  conda-forge
    curl-8.5.0                 |       hca28451_0          93 KB  conda-forge
    entrez-direct-21.6         |       he881be0_0        14.0 MB  bioconda
    gettext-0.22.5             |       h59595ed_2         464 KB  conda

Now you can use the `kraken2`. Have fun!

In [ ]:
!kraken2

Need to specify input filenames!
Usage: kraken2 [options] <filename(s)>

Options:
  --db NAME               Name for Kraken 2 DB
                          (default: none)
  --threads NUM           Number of threads (default: 1)
  --quick                 Quick operation (use first hit or hits)
  --unclassified-out FILENAME
                          Print unclassified sequences to filename
  --classified-out FILENAME
                          Print classified sequences to filename
  --output FILENAME       Print output to filename (default: stdout); "-" will
                          suppress normal output
  --confidence FLOAT      Confidence score threshold (default: 0.0); must be
                          in [0, 1].
  --minimum-base-quality NUM
                          Minimum base quality used in classification (def: 0,
                          only effective with FASTQ input).
  --report FILENAME       Print a report with aggregrate counts/clade to file
  --use-mpa-style         Wi

In [3]:
import pandas as pd

k2_viral_df = pd.read_csv('/content/viral-k2/library_report.tsv', sep='\t')
k2_viral_df['accession_id'] = k2_viral_df['Sequence Name'].apply(lambda x: x.split()[0][1:])

k2_viral_df = k2_viral_df.sort_values('accession_id')

print(k2_viral_df.shape)

(18639, 4)


In [6]:
from Bio import SeqIO

# Path to the extracted FASTA file
fasta_path = '/content/viral-ncbi/viral.1.1.genomic.fna'

# Load the FASTA file using SeqIO.parse
sequences = list(SeqIO.parse(fasta_path, "fasta"))
len(sequences)

18723

In [7]:
k2_viral_ids = k2_viral_df['accession_id'].to_list()
sequences = [seq_record for seq_record in sequences if seq_record.id in k2_viral_ids]
sequences = sorted(sequences, key=lambda x: x.id)
k2_viral_df['seq'] = [str(seq_record.seq) for seq_record in sequences]
k2_viral_df.head()

,#Library,Sequence Name,URL,accession_id,seq
15700,viral,">AC_000001.1 Ovine adenovirus A, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000001.1,CATCATCAATAATATACGGTGCATTTTGTGCGTGATGACGTATACA...
8372,viral,">AC_000002.1 Bovine adenovirus B, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000002.1,CATCATCAATAATCTACAGTACACTGATGGCAGCGGTCCAACTGCC...
1050,viral,">AC_000003.1 Canine adenovirus 1, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000003.1,CATCATCAATAATATACAGGACAAAGAGGTGTGGCCTAAATGTTGT...
12390,viral,">AC_000004.1 Duck adenovirus A, complete genome",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000004.1,CTCATGTCATTAATAAGACCATGCAGAAAATGCAAATGAGGCGAAG...
5103,viral,">AC_000005.1 Human mastadenovirus A, complete ...",ftp://ftp.ncbi.nlm.nih.gov//genomes/all/GCF/00...,AC_000005.1,CCTATCTAATAATATACCTTATACTGGACTAGTGCCAATATTAAAA...


In [10]:
k2_viral_df['seq'][0]

'...'

In [11]:
new_fasta = k2_viral_df['seq'][0]

with open('region_AA_output1.fasta', 'w') as f:
    f.write('\n'.join(new_fasta))

In [13]:
!kraken2 --db '/content/viral-k2/library_report.tsv' --output kraken_output.txt --report kraken_report.txt --threads 4 './region_AA_output1.fasta'

kraken2: database ("/content/viral-k2/library_report.tsv") does not contain necessary file taxo.k2d
